### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans, SpectralClustering, DBSCAN
from sklearn.preprocessing import OrdinalEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Part 1: Data Exploration

In [2]:
dataset = pd.read_csv('train.csv')

C:\Users\LapCell\AppData\Local\Temp\ipykernel_25644\129250805.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('train.csv')


In [3]:
print(f'Data shape: {dataset.shape}\n')
print(f'Data features: \n{dataset.dtypes}')

Data shape: (100000, 28)

Data features: 
ID                           object
Customer_ID                  object
Month                        object
Name                         object
Age                          object
SSN                          object
Occupation                   object
Annual_Income                object
Monthly_Inhand_Salary       float64
Num_Bank_Accounts             int64
Num_Credit_Card               int64
Interest_Rate                 int64
Num_of_Loan                  object
Type_of_Loan                 object
Delay_from_due_date           int64
Num_of_Delayed_Payment       object
Changed_Credit_Limit         object
Num_Credit_Inquiries        float64
Credit_Mix                   object
Outstanding_Debt             object
Credit_Utilization_Ratio    float64
Credit_History_Age           object
Payment_of_Min_Amount        object
Total_EMI_per_month         float64
Amount_invested_monthly      object
Payment_Behaviour            object
Monthly_Balance       

#### Dropping irrelevent chosen columns

In [4]:
dataset.drop(columns = ['ID','Customer_ID','Name','SSN','Type_of_Loan'], inplace = True)

#### View unique values for each column

In [5]:
# Function to help in identifying the unique values of a feature and its type to help in feature engineering
def unique_vals_and_type(data: pd.DataFrame, feature: str, cap: int):
  print(f'The unique values of \'{feature}\' feature of type {data[feature].dtype}: {dataset[feature].unique().tolist()[0:cap]}')

for feature in dataset.columns:
  unique_vals_and_type(dataset, feature, 25)

The unique values of 'Month' feature of type object: ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August']
The unique values of 'Age' feature of type object: ['23', '-500', '28_', '28', '34', '54', '55', '21', '31', '33', '34_', '7580', '30', '30_', '24', '24_', '44', '45', '40', '41', '32', '33_', '35', '35_', '36']
The unique values of 'Occupation' feature of type object: ['Scientist', '_______', 'Teacher', 'Engineer', 'Entrepreneur', 'Developer', 'Lawyer', 'Media_Manager', 'Doctor', 'Journalist', 'Manager', 'Accountant', 'Musician', 'Mechanic', 'Writer', 'Architect']
The unique values of 'Annual_Income' feature of type object: ['19114.12', '34847.84', '34847.84_', '143162.64', '30689.89', '30689.89_', '35547.71_', '35547.71', '73928.46', '131313.4', '10909427.0', '34081.38_', '34081.38', '114838.41', '114838.41_', '31370.8', '33751.27', '88640.24', '88640.24_', '54392.16', '54392.16_', '8701.545', '8701.545_', '25546.26', '25546.26_']
The unique values of 'Month

#### Transforming Features & Filtering Entries

Helper Functions

In [6]:
# Encode the Month feature to numerical values
def encode_month(month):
  calendar = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}
  return calendar.get(month)

# Removes special characters (underscores) from numeric data while ignoring nan values
def remove_underscores_numeric(data: pd.DataFrame, feature: str, remove_neg: bool = False):
  if data[feature].dtype != 'object': return
  data[feature] = pd.to_numeric(data[feature].str.replace('_', ''), errors = 'coerce')
  if remove_neg: data.loc[data[feature] < 0, feature] = np.nan

# Counts outliers which are k standard deviations away from the mean
def count_outliers(data: pd.DataFrame, feature: str, k: int):
  m,s = data[feature].mean() , data[feature].std()
  return np.sum((data[feature] < m - k*s) | (data[feature] > m + k*s))

# Clips outliers which are k standard deviations away from the mean
def clip_outliers(data: pd.DataFrame, feature: str, k: int):
  m,s = data[feature].mean() , data[feature].std()
  data[feature] = data[feature].clip(lower = m - k*s, upper = m + k*s)

Convert numerical features from 'object' type to their orignal type and replacing missing data of each with NaN.

In [7]:
dataset['Month'] = dataset['Month'].apply(encode_month).astype(int)
remove_underscores_numeric(dataset, 'Age', remove_neg = True)
remove_underscores_numeric(dataset, 'Num_of_Loan', remove_neg = True)
remove_underscores_numeric(dataset, 'Num_of_Delayed_Payment')
dataset.loc[dataset['Num_Bank_Accounts'] < 0, 'Num_Bank_Accounts'] = np.nan
dataset['Num_Credit_Inquiries'] = pd.to_numeric(dataset['Num_Credit_Inquiries'], errors = 'coerce')
remove_underscores_numeric(dataset, 'Annual_Income')
remove_underscores_numeric(dataset, 'Changed_Credit_Limit')
remove_underscores_numeric(dataset, 'Outstanding_Debt')
remove_underscores_numeric(dataset, 'Amount_invested_monthly')
remove_underscores_numeric(dataset, 'Monthly_Balance')
remove_underscores_numeric(dataset, 'Num_of_Delayed_Payment')

Cleaning string data and converting them to encoded categorical variables. 

In [ ]:
dataset['Occupation'] = dataset['Occupation'].replace(to_replace = '_______', value = np.nan)
dataset['Credit_Mix'] = dataset['Credit_Mix'].replace(to_replace = '_', value = np.nan)
# Replace the Credit History Age column with years and months.
temp_df = dataset['Credit_History_Age'].str.extract(r'(?P<Years>[\d]?[\d]) Years and (?P<Months>[\d]?[\d]) Months', expand = True)
dataset['Credit_History_Age'] = temp_df['Years'].astype('Int64') + (temp_df['Months'].astype('Int64') / 12)
dataset['Payment_of_Min_Amount'] = dataset['Payment_of_Min_Amount'].replace(to_replace = 'NM', value = np.nan)
dataset['Payment_Behaviour'] = dataset['Payment_Behaviour'].replace(to_replace = '!@9#%8', value = np.nan)

In [ ]:
# Encoding categorical variables
occupation_encoder = OrdinalEncoder()
dataset['Occupation'] = occupation_encoder.fit_transform(dataset['Occupation'].values.reshape(-1,1))

Describe the dataset to see summaries of each feature.

In [ ]:
dataset.describe()

Dealing with missing values and outliers

In [ ]:
# Count outliers and NA values for each numerical feature and store results in a dataframe
features, outliers, percent_outliers, nas, percent_nas = [],[],[],[],[]

for f in dataset.columns:
  features.append(f)
  if (dataset[f].dtype != 'object'):
    outliers.append(count_outliers(dataset, f, 3))
    percent_outliers.append(outliers[-1]/dataset.shape[0] * 100)
  else:
    outliers.append(np.nan)
    percent_outliers.append(np.nan)
  nas.append(dataset[f].isna().sum())
  percent_nas.append(nas[-1]/dataset.shape[0] * 100)

pd.DataFrame({'Feature': features, 'Outliers': outliers, 'Outlier%': percent_outliers, 'NA': nas, 'NA%': percent_nas})

All outliers seem to be within the 5% range of the whole dataset, so we will leave them as they might belong to the original distribution. On the other hand, NA values will cause problems, and will incur a noticable loss of information if we just delete the rows or columns which have NA values, so we will use data imputation by iterative multivariate imputation.

In [ ]:
imputer = IterativeImputer(max_iter = 10, random_state = 42)
imputed_data = imputer.fit_transform(dataset)
dataset = pd.DataFrame(imputed_data, columns = dataset.columns)